In [1]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
import json



root_url = "https://app.ticketmaster.com/discovery/v2/"

In [2]:
def desde_fecha(fecha_str):
        """
        Convierte 'YYYY-MM-DD' a 'YYYY-MM-DDTHH:MM:SSZ'
        """
        return f'{fecha_str}T00:00:00Z'

In [3]:
def hasta_fecha(fecha_str):
    """
    Convierte 'YYYY-MM-DD' a 'YYYY-MM-DDTHH:MM:SSZ' (Final del día)
    """
    return f'{fecha_str}T23:59:59Z'

In [4]:
def extraccion_actual(ciudad, fecha_inicio, API_KEY, fecha_fin, pais='US'):

    url = f'{root_url}events.json'

    params = {
        'apikey' : API_KEY,
        'city' : ciudad,
        'countryCode' : pais,
        'startDateTime': fecha_inicio,
        'endDateTime': fecha_fin
    }

    response = requests.get(url, params=params)

    assert response.status_code == 200, "Error en la extracción"
    data = response.json()
    if '_embedded' not in data or 'events' not in data['_embedded']:
        print("No se encontraron eventos")
        return pd.DataFrame()
    
    eventos = data['_embedded']['events']
    
    df = pd.DataFrame(eventos)
    
    return df

In [5]:
API_KEY = os.getenv('TICKETMASTER_API_KEY')
assert API_KEY is not None, "Falta la variable de entorno TICKETMASTER_API_KEY"
fecha_actual = desde_fecha(datetime.now().strftime('%Y-%m-%d'))
fecha_fin = hasta_fecha(datetime.now().strftime('%Y-%m-%d'))

df = extraccion_actual('New York', fecha_actual, API_KEY, fecha_fin)
df.loc[0]

name                                                   Hamilton (NY)
type                                                           event
id                                               Z1r9uZrrZbpZ1AvjVus
test                                                           False
url                https://www.ticketmaster.com/hamilton-ny-new-y...
locale                                                         en-us
images             [{'ratio': '3_2', 'url': 'https://s1.ticketm.n...
sales              {'public': {'startDateTime': '2025-07-31T14:00...
dates              {'start': {'localDate': '2026-02-14', 'localTi...
classifications    [{'primary': True, 'segment': {'id': 'KZFzniwn...
promoter           {'id': '301', 'name': 'NEDERLANDER PRODUCTIONS...
promoters          [{'id': '301', 'name': 'NEDERLANDER PRODUCTION...
ticketLimit        {'info': 'The established ticket limit is 8 ti...
_links             {'self': {'href': '/discovery/v2/events/Z1r9uZ...
_embedded          {'venues': [{'n

In [6]:
df.columns

Index(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'sales',
       'dates', 'classifications', 'promoter', 'promoters', 'ticketLimit',
       '_links', '_embedded', 'outlets', 'seatmap', 'ticketing', 'nameOrigin',
       'pleaseNote', 'products', 'accessibility', 'ageRestrictions', 'info',
       'description'],
      dtype='object')

In [7]:
df.loc[0, "sales"]

{'public': {'startDateTime': '2025-07-31T14:00:00Z',
  'startTBD': False,
  'startTBA': False,
  'endDateTime': '2026-02-14T17:00:00Z'}}

In [8]:
df = df[["name",  "dates", "_embedded"]]
df.loc[15, "dates"]

{'access': {'startDateTime': '2025-08-06T17:24:34Z',
  'startApproximate': False,
  'endDateTime': '2026-02-14T22:30:00Z',
  'endApproximate': False},
 'start': {'localDate': '2026-02-14',
  'localTime': '15:30:00',
  'dateTime': '2026-02-14T20:30:00Z',
  'dateTBD': False,
  'dateTBA': False,
  'timeTBA': False,
  'noSpecificTime': False},
 'end': {'localTime': '15:30:00',
  'dateTime': '2026-02-14T22:30:00Z',
  'approximate': False,
  'noSpecificTime': False},
 'timezone': 'America/New_York',
 'status': {'code': 'onsale'},
 'spanMultipleDays': False}

In [9]:

df['hora_inicio'] = df['dates'].apply(lambda x: x['start']['dateTime'])
df['hora_final'] = df['dates'].apply(lambda x: x['end']['dateTime'] if 'end' in x else None)
df['direccion'] = df["_embedded"].apply(lambda val: val["venues"][0]["address"]["line1"])
df['CP'] = df["_embedded"].apply(lambda val: val["venues"][0]["postalCode"])
df['parking'] = df["_embedded"].apply(lambda val: "Yes" if "parkingDetail" in val["venues"][0] else "No")
df = df.drop("dates", axis = 1)
df = df.drop("_embedded", axis=1)
df

,name,hora_inicio,hora_final,direccion,CP,parking
0,Hamilton (NY),2026-02-14T18:00:00Z,None,226 West 46th Street,10036,Yes
1,Harry Potter and the Cursed Child,2026-02-14T19:00:00Z,None,213 W 42nd St,10036,No
2,Hamilton,2026-02-14T18:00:00Z,None,226 W. 46th St.,10036,No
3,The Office! A Musical Parody,2026-02-14T21:00:00Z,None,210 W. 50th Street,10019,Yes
4,The Disappear,2026-02-14T19:00:00Z,None,18 Minetta Lane,10012,Yes
5,11 to Midnight,2026-02-14T20:00:00Z,None,126 Second Ave.,10003,Yes
6,Perfect Crime,2026-02-14T19:00:00Z,None,210 W. 50th Street,10019,Yes
7,Haus of Hard Rock Fashion Show,2026-02-14T15:00:00Z,None,1501 Broadway,10036,No
8,Singfeld! A Musical About Nothing!,2026-02-14T23:30:00Z,None,210 W. 50th Street,10019,Yes
9,The Banksy Museum New York!,2026-02-14T21:00:00Z,2026-02-14T23:00:00Z,277 Canal St,10013,No


In [10]:
df['hora_inicio'] = pd.to_datetime((df['hora_inicio']))
df['hora_final'] = pd.to_datetime((df['hora_final']))

In [11]:
df['hora_inicio'] = df['hora_inicio'].dt.tz_convert('America/New_York')
df['hora_final'] = df['hora_final'].dt.tz_convert('America/New_York')

In [12]:
df['hora_inicio'] = df['hora_inicio'].dt.strftime('%H:%M')
df['hora_final'] = df['hora_final'].dt.strftime('%H:%M')

In [13]:
df.head(5)

,name,hora_inicio,hora_final,direccion,CP,parking
0,Hamilton (NY),13:00,NaN,226 West 46th Street,10036,Yes
1,Harry Potter and the Cursed Child,14:00,NaN,213 W 42nd St,10036,No
2,Hamilton,13:00,NaN,226 W. 46th St.,10036,No
3,The Office! A Musical Parody,16:00,NaN,210 W. 50th Street,10019,Yes
4,The Disappear,14:00,NaN,18 Minetta Lane,10012,Yes
